# Импорт

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LookGenerator.datasets.person_segmantation_dataset import PersonSegmentationDataset, PersonSegmentationDatasetMultichannel
from LookGenerator.networks.segmentation import UNet, train_unet
from LookGenerator.networks.utils import load_model

# Загрузка данных

In [2]:
transform_input = transforms.Compose([
    transforms.Resize((192, 256)),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.25, 0.25, 0.25]
    )
])

transform_output = transforms.Compose([
    transforms.Resize((192, 256)),
    transforms.Normalize(
        mean=[0.5],
        std=[0.25]
    )
])

In [3]:
batch_size_train = 12
batch_size_val = 8

In [4]:
train_dataset = PersonSegmentationDatasetMultichannel("C:\\Users\\DenisovDmitrii\\Desktop\\train", transform_input=transform_input, transform_mask=transform_output)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size_train, shuffle=True)
len(train_dataloader)

270

In [5]:
val_dataset = PersonSegmentationDatasetMultichannel("C:\\Users\\DenisovDmitrii\\Desktop\\test", transform_input=transform_input, transform_mask=transform_output)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size_val, shuffle=False)
len(val_dataloader)

103

Test load train_dataloader. and val.

In [6]:
for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([12, 3, 192, 256])
torch.Size([12, 16, 192, 256])


# Обучение модели

Метки классов на изображении:

1. Левое лицо                       248,251,14
2. Правое лицо                      251,235,25
3. Туловище                         20,80,194
4. Внешняя верхняя левая рука       190,189,96
5. Внешняя верхняя правая рука      215,187,88
6. Внешняя нижняя левая рука        252,206,48
7. Внешняя нижняя правая рука       250,220,36
8. Внутренняя верхняя левая рука    145,191,116
9. Внутренняя верхняя правая рука   170,190,105
10. Внетренняя нижняя левая рука     228,191,74
11. Внутренняя нижняя правая рука    240,198,60
12. Левая кисть                      8,110,221
13. Правая кисть                     4,98,224
14. Левая нога                       22,173,184
15. Правая нога                      6,166,198

In [2]:
model = UNet(in_channels=3, out_channels = 16)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [8]:
import time
startTime = time.time()

train_history, val_history = train_unet(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    device=device,
    epoch_num=10,
    save_directory="C:\\Users\\DenisovDmitrii\\Desktop\\segmentation_weits\\session5"
)

print(time.time() - startTime)


Epoch 0 of 9, train loss: 0.004
Epoch 0 of 9, val loss: 0.016
Epoch 1 of 9, train loss: 0.000
Epoch 1 of 9, val loss: 0.028
Epoch 2 of 9, train loss: 0.000
Epoch 2 of 9, val loss: 0.019
Epoch 3 of 9, train loss: 0.000
Epoch 3 of 9, val loss: 0.041
Epoch 4 of 9, train loss: 0.000
Epoch 4 of 9, val loss: 0.025
Epoch 5 of 9, train loss: 0.000
Epoch 5 of 9, val loss: 0.042
Epoch 6 of 9, train loss: 0.000
Epoch 6 of 9, val loss: 0.029
Epoch 7 of 9, train loss: 0.000
Epoch 7 of 9, val loss: 0.027
Epoch 8 of 9, train loss: 0.000
Epoch 8 of 9, val loss: 0.033
Epoch 9 of 9, train loss: 0.000
Epoch 9 of 9, val loss: 0.039
7209.106112718582


In [3]:
model = load_model(model, r'C:\Users\Даша\PycharmProjects\SMBackEnd\LookGenerator\weights\unet_epoch_0_0.0161572862694324.pt')
model.eval()

RuntimeError: Error(s) in loading state_dict for UNet:
	Missing key(s) in state_dict: "ups.1.net.0.net.1.weight", "ups.1.net.0.net.1.bias", "ups.1.net.0.net.1.running_mean", "ups.1.net.0.net.1.running_var", "ups.1.net.1.net.1.weight", "ups.1.net.1.net.1.bias", "ups.1.net.1.net.1.running_mean", "ups.1.net.1.net.1.running_var", "ups.3.net.0.net.1.weight", "ups.3.net.0.net.1.bias", "ups.3.net.0.net.1.running_mean", "ups.3.net.0.net.1.running_var", "ups.3.net.1.net.1.weight", "ups.3.net.1.net.1.bias", "ups.3.net.1.net.1.running_mean", "ups.3.net.1.net.1.running_var", "ups.5.net.0.net.1.weight", "ups.5.net.0.net.1.bias", "ups.5.net.0.net.1.running_mean", "ups.5.net.0.net.1.running_var", "ups.5.net.1.net.1.weight", "ups.5.net.1.net.1.bias", "ups.5.net.1.net.1.running_mean", "ups.5.net.1.net.1.running_var", "ups.7.net.0.net.1.weight", "ups.7.net.0.net.1.bias", "ups.7.net.0.net.1.running_mean", "ups.7.net.0.net.1.running_var", "ups.7.net.1.net.1.weight", "ups.7.net.1.net.1.bias", "ups.7.net.1.